In [ ]:
# import praw
# import pandas as pd
# import time
# reddit = praw.Reddit(client_id='FPPSnsUDHnlYYQv2AxR82Q',client_secret='EoUTTbJI3sIcW_yBr0KipdDHEdOxpQ',
# user_agent='your_user_agent')# Topic to search

# topic = 'menopause' # Subreddit to search


# arr1 = []
# arr2 = []

# itr = 1
# subreddit = reddit.subreddit(topic)  # Get the top 100 posts in the last month for the given topic
# posts = subreddit.search(topic, time_filter='month', limit=2) # Extract the content of each post
# for post in posts:
# #     time.sleep(2)
#     arr1.append([post.title, post.score, post.id, post.url, post.num_comments,post.permalink ,post.selftext, post.upvote_ratio,post.created])
#     post.comments.replace_more(limit=0)
#     for comment in post.comments.list():
#         arr2.append([post.id,comment.body])
# #             print(comment.body)
# #             a.append(comment.body)
#     print(itr)
#     itr = itr + 1
# arr1 = pd.DataFrame(arr1,columns=['title', 'score', 'id', 'url', 'num_comments','permalink', 'body', 'upvote_ratio' , 'created'])
# arr2 = pd.DataFrame(arr2,columns=['id','comment'])
    
    

In [ ]:
# arr1.to_csv('a.csv',index=False)

In [ ]:
# arr2.to_csv('b.csv',index=False)

In [41]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import numpy as np
from tqdm import tqdm
from emot.emo_unicode import UNICODE_EMOJI, UNICODE_EMOJI_ALIAS, EMOTICONS_EMO
from bs4 import BeautifulSoup
from flashtext import KeywordProcessor
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ilovemarijuana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ilovemarijuana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ilovemarijuana/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ilovemarijuana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df = pd.read_csv('comments.csv')

In [3]:
df

,id,comment
0,10tl0s7,"So I'm in Peri, but so far I've noticed things..."
1,10tl0s7,Just letting my hair go grey. I used to be a g...
2,10tl0s7,My SO and I started going to raves about 4 yea...
3,10tl0s7,Love those shoes! When I turned 50 I gifted m...
4,10tl0s7,I’ve stopped carrying a purse. Which means I c...
...,...,...
9056,10shrwt,"My doc increased my progesterone this week, so..."
9057,10shrwt,"Yes, I am and that includes 200 mg of progeste..."
9058,10shrwt,"Yeah, I had mine increased to 200 / night - ze..."
9059,10shrwt,"Shoot, I was afraid you were going to say that..."


In [4]:
# Code for removing emojis: 
# I am commenting it out because whole process takes huge time.
all_reviews_without_emot = []

def cleaning_emot(review,i):
    all_emoji_emoticons = {**EMOTICONS_EMO,**UNICODE_EMOJI_ALIAS, **UNICODE_EMOJI_ALIAS}
    all_emoji_emoticons = {k:v.replace(":","").replace("_"," ").strip() for k,v in all_emoji_emoticons.items()}
    letters = BeautifulSoup(review).get_text()
    kp_all_emoji_emoticons = KeywordProcessor()
    for k,v in all_emoji_emoticons.items():
        kp_all_emoji_emoticons.add_keyword(k, v)
    try:
        extracted_emoji_text = kp_all_emoji_emoticons.replace_keywords(review)
        all_reviews_without_emot.append(extracted_emoji_text)
    except:
        all_reviews_without_emot.append(letters)

In [5]:
i = 0
for review in tqdm(df['comment']):
    temp = cleaning_emot(review,i)

  0%|                                        | 15/9061 [00:00<01:01, 147.95it/s]/opt/anaconda3/envs/ta/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
 10%|███▉                                   | 924/9061 [00:06<00:57, 140.82it/s]/opt/anaconda3/envs/ta/lib/python3.10/site-packages/bs4/__init__.py:404: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
100%|██████████████████████████████████████| 9061/9061 [01:00<00:00, 148.75it/s]


In [6]:
stop_words = set(stopwords.words('english'))
def clean(review):
    letters_only = BeautifulSoup(review).get_text()
    letters_only = re.sub('[^a-zA-Z]', ' ', letters_only)
    letters_only = letters_only.lower()
    words = letters_only.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

In [7]:
all_reviews = []
for review in tqdm(all_reviews_without_emot):
    all_reviews.append(clean(review))

100%|█████████████████████████████████████| 9061/9061 [00:01<00:00, 5213.02it/s]


In [11]:
all_reviews[2]

'started going raves years ago often oldest among oldest attendance previously would self conscious public stand care anymore blast'

In [17]:
lem = WordNetLemmatizer()
final_reviews = []
def Lemmatizer(review):
    lem_words = []
    tokenized_review = nltk.word_tokenize(review)
    for word in tokenized_review:
        lem_words.append(lem.lemmatize(word))
    return " ".join(lem_words)

In [18]:
final_reviews = []
for review in tqdm(all_reviews):
    final_reviews.append(Lemmatizer(review))

100%|█████████████████████████████████████| 9061/9061 [00:02<00:00, 3023.15it/s]


In [19]:
final_reviews

['peri far noticed thing bother much like idgaf finally hated little thing eating inside telling honestly honey real good move btw love kick si rock em',
 'letting hair go grey used goth make beautiful clothes boot old lady ripe old age rock night gig care want quiet life biggest ambition getting good night sleep without tossing turning night',
 'started going raf year ago often oldest among oldest attendance previously would self conscious public stand care anymore blast',
 'love shoe turned gifted first tattoo used think never want permanent art recently got new ear piercings reaching age want going pretty awesome',
 'stopped carrying purse mean can not wear anything pocket small tiny wallet like three card includes id use one rfid blocking sleeve key phone one ever ask carry something bwahahaha also freedom knowing never lose purse nice found without purse stride purposeful sure really like combined murder face people scurry way',
 'loving every inch forgiving younger thought best t

In [29]:
tfidfvectorizer = TfidfVectorizer(analyzer='word', max_features=500)
reviews_vect = tfidfvectorizer.fit_transform(final_reviews)
reviews_vect = reviews_vect.toarray()

In [31]:
len(reviews_vect)

9061

In [49]:
k = 100

In [50]:
def get_silhouette_score(X, k):
    for n_clusters in range(2, k):
        clusterer = KMeans(init="k-means++", n_clusters=n_clusters, random_state=42)
        y = clusterer.fit_predict(X)

        message = "For n_clusters = {} The average silhouette_score is: {}"
        print(message.format(n_clusters, silhouette_score(X, y)))

get_silhouette_score(reviews_vect, k)

/opt/anaconda3/envs/ta/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 2 The average silhouette_score is: 0.011547621248707621


/opt/anaconda3/envs/ta/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 3 The average silhouette_score is: 0.01729350433783352


/opt/anaconda3/envs/ta/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


...

For n_clusters = 97 The average silhouette_score is: 0.048758760889403334


/opt/anaconda3/envs/ta/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 98 The average silhouette_score is: 0.04995846224982725


/opt/anaconda3/envs/ta/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


For n_clusters = 99 The average silhouette_score is: 0.05147533911011686


In [51]:
k = 54

In [52]:
model = KMeans(init="k-means++", n_clusters=k, max_iter=25, n_init=1)
model.fit(reviews_vect)

KMeans(max_iter=25, n_clusters=54, n_init=1)

In [54]:
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = tfidfvectorizer.get_feature_names_out()

results_dict = {}

for i in range(k):
    terms_list = []

    for ind in order_centroids[i, :]:  
        terms_list.append(terms[ind])

    results_dict[f'Cluster {i}'] = terms_list

df_clusters = pd.DataFrame.from_dict(results_dict)
df_clusters

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,...,Cluster 44,Cluster 45,Cluster 46,Cluster 47,Cluster 48,Cluster 49,Cluster 50,Cluster 51,Cluster 52,Cluster 53
0,omg,nice,option,thing,face,brand,yes,helpful,know,article,...,good,use,book,love,yeah,omg,http,name,right,cancer
1,soon,care,others,turn,smiley,know,oh,thank,happening,menopause,...,luck,cream,menopause,shoe,oh,laugh,com,karen,got,breast
2,place,old,menopause,fun,happy,like,one,thanks,looking,http,...,one,work,dr,idea,like,love,www,lol,year,hrt
3,big,every,bad,might,thank,use,get,much,young,woman,...,know,get,read,much,get,thanks,reddit,someone,know,risk
4,mine,right,even,could,eye,buy,make,really,everyone,read,...,thank,also,check,would,doctor,hard,menopause,actually,day,menopause
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,put,product,problem,point,helpful,perimenopause,bad,never,product,men,...,super,talk,thought,tried,without,least,though,pretty,smell,hour
496,question,progesterone,product,possible,sex,something,please,week,progesterone,luck,...,pill,guess,symptom,need,hope,put,mine,menopause,doc,thanks
497,quite,put,progesterone,post,night,every,point,used,put,mine,...,smell,bed,hormone,every,hot,estrogen,omg,hrt,find,world
498,read,question,put,prescribed,mine,want,second,put,question,going,...,doc,point,went,thing,week,progesterone,idea,happy,without,hear
